# O exemplo da altura e peso médios

As duas células abaixo foram extraídas do [repositório do livro](https://github.com/fluentpython/example-code/blob/master/16-coroutine/coroaverager3.py).

In [44]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

# the subgenerator
def averager():  # <1>
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield  # <2>
        if term is None:  # <3>
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)  # <4>


# the delegating generator
def grouper(results, key):  # <5>
    while True:  # <6>
        results[key] = yield from averager()  # <7>


# the client code, a.k.a. the caller
def main(data):  # <8>
    results = {}
    for key, values in data.items():
        group = grouper(results, key)  # <9>
        next(group)  # <10>
        for value in values:
            group.send(value)  # <11>
        group.send(None)  # important! <12>

    # print(results)  # uncomment to debug
    report(results)


# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))


In [45]:
data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


## A minha versão

A planilha `08_exemplo.csv` tem a medida gerada por IA dos pesos e alturas de meninos e meninas fictícios.

In [67]:
Media = namedtuple('Media', 'soma contagem media')

# subgerador
def calcula_media():
    total = 0.0
    contagem = 0
    termo = yield
    while termo is not None:
        total += termo
        contagem += 1
        termo = yield
    return Media(round(total, 2), contagem, round(total / max(contagem, 1.0), 2))


# delegante
def agrupador(medias, chave):
    while True:
        medias[chave] = yield from calcula_media()


# chamador
def chamador():
    with open('08_medidas.csv', 'r', encoding='utf8') as file:
        arquivo = [x.strip().split(',') for x in file.readlines()[1:]]

    data = {}
    for linha in arquivo:
        genero = linha[0]
        if f'{genero};m' not in data:
            data[f'{genero};m'] = []
        if f'{genero};kg' not in data:
            data[f'{genero};kg'] = []
        data[f'{genero};m'].append(float(linha[1]))
        data[f'{genero};kg'].append(float(linha[2]))
    

    medias = {}
    for chave, valor in data.items():
        grupo = agrupador(medias, chave)
        next(grupo)
        for value in valor:
            grupo.send(value)
        grupo.send(None)  # important!

    for chave in medias:
        print(chave, medias[chave])

In [68]:
chamador()

boy;m Media(soma=2904.03, contagem=19, media=152.84)
boy;kg Media(soma=1159.54, contagem=19, media=61.03)
girl;m Media(soma=2966.24, contagem=19, media=156.12)
girl;kg Media(soma=1192.65, contagem=19, media=62.77)
